In [2]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv("local.env")

# Access the API key
api_key = os.getenv("OPEN_AI_API_KEY")

In [4]:
import requests

# Function to fetch PR details including the diff
def fetch_pr_diff(owner, repo, pr_number, github_token=None):
    url = f"https://api.github.com/repos/{owner}/{repo}/pulls/{pr_number}"
    headers = {"Accept": "application/vnd.github.v3.diff"}
    if github_token:
        headers["Authorization"] = f"token {github_token}"
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.text  # Returns the diff of the PR
    else:
        print("Failed to fetch PR details. Status Code:", response.status_code)
        return None

# Function to simulate sending the diff to GPT-4 for review
def review_code_with_gpt4(code_diff):
    # Simulate sending the code diff to GPT-4 and receiving feedback
    # Replace this with actual OpenAI API call
    simulated_feedback = "The code changes look good overall. However, consider adding error handling for the API call."
    return simulated_feedback

# Main function to orchestrate the PR review process
def main():
    owner = "wkda"  # GitHub repository owner
    repo = "ds-auto-pricing-service"  # Repository name
    pr_number = 1231  # PR number
    github_token = None  # Your GitHub Personal Access Token, if needed

    print(f"Fetching PR #{pr_number} diff from {owner}/{repo}...")
    pr_diff = fetch_pr_diff(owner, repo, pr_number, github_token)
    
    if pr_diff:
        print("Reviewing code changes with GPT-4...")
        feedback = review_code_with_gpt4(pr_diff)
        print("\nGPT-4 Feedback:\n", feedback)


In [5]:
owner = "wkda"  # GitHub repository owner
repo = "ds-auto-pricing-service"  # Repository name
pr_number = 1231  # PR number
github_token = os.getenv("GITHUB_API_KEY")  # Your GitHub Personal Access Token, if needed

In [6]:
fetch_pr_diff(owner, repo, pr_number, github_token=github_token)

'diff --git a/notebooks/check_ap_car_info.ipynb b/notebooks/check_ap_car_info.ipynb\nindex 905fbee45..90e407a79 100644\n--- a/notebooks/check_ap_car_info.ipynb\n+++ b/notebooks/check_ap_car_info.ipynb\n@@ -1,7 +1,6 @@\n {\n  "cells": [\n   {\n-   "attachments": {},\n    "cell_type": "markdown",\n    "metadata": {},\n    "source": [\n@@ -9,7 +8,6 @@\n    ]\n   },\n   {\n-   "attachments": {},\n    "cell_type": "markdown",\n    "metadata": {},\n    "source": [\n@@ -17,7 +15,6 @@\n    ]\n   },\n   {\n-   "attachments": {},\n    "cell_type": "markdown",\n    "metadata": {},\n    "source": [\n@@ -26,7 +23,7 @@\n     "- check pricing history table\\n",\n     "- load the prod models from s3 to local\\n",\n     "- prediction with the loaded models\\n",\n-    "- compare current API data with private_listing data\\n",\n+    "- compare current API data with self_evaluations data\\n",\n     "- check dynamic pricing\\n",\n     "- check change log data\\n",\n     "- predict with your set of data\\n"

In [9]:
from openai import OpenAI

def review_code_with_gpt4(code_diff, openai_api_key):
    """
    Sends the code diff to GPT-4 and receives feedback.
    
    :param code_diff: The code differences as a string.
    :param openai_api_key: Your OpenAI API key.
    :return: The feedback from GPT-4 as a string.
    """
    client = OpenAI(api_key = openai_api_key)
    try:
        response = client.chat.completions.create(
            model="gpt-4",  # Use the latest available model
            messages=[
                {
                    "role":"user", 
                    "content":f"Review the following code changes:\n\n{code_diff}\n\nProvide detailed feedback:"
                }
            ]
        )
        return response.choices[0].text.strip()
    except Exception as e:
        print(f"An error occurred: {e}")
        return "Failed to get feedback from GPT-4."

# Main function adjustment to include the OpenAI API key parameter
def main():
    owner = "wkda"
    repo = "ds-auto-pricing-service"
    pr_number = 1231
    github_token = os.getenv("GITHUB_API_KEY")  # Replace with your GitHub token if needed
    openai_api_key = os.getenv("OPEN_AI_API_KEY")  # Replace with your OpenAI API key

    print(f"Fetching PR #{pr_number} diff from {owner}/{repo}...")
    pr_diff = fetch_pr_diff(owner, repo, pr_number, github_token)
    
    if pr_diff:
        print("Reviewing code changes with GPT-4...")
        feedback = review_code_with_gpt4(pr_diff, openai_api_key)
        print("\nGPT-4 Feedback:\n", feedback)

if __name__ == "__main__":
    main()


Fetching PR #1231 diff from wkda/ds-auto-pricing-service...
Reviewing code changes with GPT-4...
An error occurred: Error code: 400 - {'error': {'message': "This model's maximum context length is 8192 tokens. However, your messages resulted in 12988 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}

GPT-4 Feedback:
 Failed to get feedback from GPT-4.


In [12]:
def split_diff_and_review(code_diff, openai_api_key, max_tokens=7000):
    # Assuming `code_diff` is a list of lines
    lines = code_diff.split('\n')
    feedbacks = []

    # Splitting the diff into chunks
    for i in range(0, len(lines), max_tokens):
        chunk = '\n'.join(lines[i:i+max_tokens])
        feedback = review_code_with_gpt4(chunk, openai_api_key)
        feedbacks.append(feedback)

    return " ".join(feedbacks)


In [13]:
owner = "wkda"
repo = "ds-auto-pricing-service"
pr_number = 1231
github_token = os.getenv("GITHUB_API_KEY")  # Replace with your GitHub token if needed
openai_api_key = os.getenv("OPEN_AI_API_KEY")  # Replace with your OpenAI API key

print(f"Fetching PR #{pr_number} diff from {owner}/{repo}...")
pr_diff = fetch_pr_diff(owner, repo, pr_number, github_token)
split_diff_and_review(pr_diff, openai_api_key, max_tokens=7000)


Fetching PR #1231 diff from wkda/ds-auto-pricing-service...
An error occurred: Error code: 400 - {'error': {'message': "This model's maximum context length is 8192 tokens. However, your messages resulted in 12988 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}


'Failed to get feedback from GPT-4.'

In [17]:
import openai

def review_code_with_gpt4(code_diff, openai_api_key):
    """
    Sends the code diff to GPT-4 and receives feedback.
    
    :param code_diff: The code differences as a string.
    :param openai_api_key: Your OpenAI API key.
    :return: The feedback from GPT-4 as a string.
    """
    client = OpenAI(api_key = openai_api_key)
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {
                    "role":"user", 
                    "content":f"Review the following code changes:\n\n{code_diff}\n\nProvide detailed feedback:"
                }
            ]
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"An error occurred: {e}")
        return "Failed to get feedback from GPT-4."

def split_diff_and_review(code_diff, openai_api_key, max_chars=8000):
    """
    Splits the diff into chunks based on a character count approximation to stay within token limits.
    """
    # Initialize variables
    feedbacks = []
    chunk = ""

    # Splitting the diff more accurately by character count
    for line in code_diff.split('\n'):
        # Check if adding the next line would exceed the max character count
        if len(chunk) + len(line) + 1 > max_chars:  # +1 for newline character
            # Review the current chunk
            feedback = review_code_with_gpt4(chunk, openai_api_key)
            feedbacks.append(feedback)
            # Start a new chunk
            chunk = line
        else:
            # Add the line to the current chunk
            chunk += f"\n{line}"

    # Don't forget to process the last chunk if it's not empty
    if chunk:
        feedback = review_code_with_gpt4(chunk, openai_api_key)
        feedbacks.append(feedback)

    return " ".join(feedbacks)

# Example usage
owner = "wkda"
repo = "ds-auto-pricing-service"
pr_number = 1231
github_token = os.getenv("GITHUB_API_KEY")  # Replace with your GitHub token if needed
openai_api_key = os.getenv("OPEN_AI_API_KEY")  # Replace with your OpenAI API key

print(f"Fetching PR #{pr_number} diff from {owner}/{repo}...")
pr_diff = fetch_pr_diff(owner, repo, pr_number, github_token)
feedback = split_diff_and_review(pr_diff, openai_api_key)
print(feedback)


Fetching PR #1231 diff from wkda/ds-auto-pricing-service...
1. In the first code change, lines 2-6 have been removed. These lines contained cell attachments, which were likely unnecessary. It's good to remove unnecessary code to improve readability and reduce unnecessary code execution.

2. In the second code change, lines 8-10 have been modified. The code was previously loading an environment file using the "load_dotenv_to_environ" function. The code now includes a line break before calling the function. This change improves readability by separating the function call from the import statement.

3. In the third code change, lines 12-14 have been modified. The code was previously setting display options for pandas, but the options were not being used in the code. The code now includes a line break before setting the display options. This change improves readability by separating the display options from the following code.

4. In the fourth code change, lines 36-40 have been modified. 

In [18]:
pr_diff

'diff --git a/notebooks/check_ap_car_info.ipynb b/notebooks/check_ap_car_info.ipynb\nindex 905fbee45..90e407a79 100644\n--- a/notebooks/check_ap_car_info.ipynb\n+++ b/notebooks/check_ap_car_info.ipynb\n@@ -1,7 +1,6 @@\n {\n  "cells": [\n   {\n-   "attachments": {},\n    "cell_type": "markdown",\n    "metadata": {},\n    "source": [\n@@ -9,7 +8,6 @@\n    ]\n   },\n   {\n-   "attachments": {},\n    "cell_type": "markdown",\n    "metadata": {},\n    "source": [\n@@ -17,7 +15,6 @@\n    ]\n   },\n   {\n-   "attachments": {},\n    "cell_type": "markdown",\n    "metadata": {},\n    "source": [\n@@ -26,7 +23,7 @@\n     "- check pricing history table\\n",\n     "- load the prod models from s3 to local\\n",\n     "- prediction with the loaded models\\n",\n-    "- compare current API data with private_listing data\\n",\n+    "- compare current API data with self_evaluations data\\n",\n     "- check dynamic pricing\\n",\n     "- check change log data\\n",\n     "- predict with your set of data\\n"

In [19]:
pr_diff.split("\n")

['diff --git a/notebooks/check_ap_car_info.ipynb b/notebooks/check_ap_car_info.ipynb',
 'index 905fbee45..90e407a79 100644',
 '--- a/notebooks/check_ap_car_info.ipynb',
 '+++ b/notebooks/check_ap_car_info.ipynb',
 '@@ -1,7 +1,6 @@',
 ' {',
 '  "cells": [',
 '   {',
 '-   "attachments": {},',
 '    "cell_type": "markdown",',
 '    "metadata": {},',
 '    "source": [',
 '@@ -9,7 +8,6 @@',
 '    ]',
 '   },',
 '   {',
 '-   "attachments": {},',
 '    "cell_type": "markdown",',
 '    "metadata": {},',
 '    "source": [',
 '@@ -17,7 +15,6 @@',
 '    ]',
 '   },',
 '   {',
 '-   "attachments": {},',
 '    "cell_type": "markdown",',
 '    "metadata": {},',
 '    "source": [',
 '@@ -26,7 +23,7 @@',
 '     "- check pricing history table\\n",',
 '     "- load the prod models from s3 to local\\n",',
 '     "- prediction with the loaded models\\n",',
 '-    "- compare current API data with private_listing data\\n",',
 '+    "- compare current API data with self_evaluations data\\n",',
 '     "- ch

In [20]:
def filter_diff_for_sql_and_py(diff_lines):
    """
    Filters out diffs from .ipynb files, focusing on .sql and .py files.

    :param diff_lines: A list of lines from the diff.
    :return: A list of filtered lines, excluding .ipynb file diffs.
    """
    filtered_diff = []
    skip_chunk = False

    for line in diff_lines:
        if line.startswith('diff --git'):
            # Check if this diff chunk is for an .ipynb file
            skip_chunk = '.ipynb' in line
        if not skip_chunk:
            filtered_diff.append(line)

    return filtered_diff

# Example usage
diff_lines = [
    'diff --git a/notebooks/check_ap_car_info.ipynb b/notebooks/check_ap_car_info.ipynb',
    'index 905fbee45..90e407a79 100644',
    '--- a/notebooks/check_ap_car_info.ipynb',
    '+++ b/notebooks/check_ap_car_info.ipynb',
    '@@ -1,7 +1,6 @@',
    ' {',
    '  "cells": [',
    '   {',
    '-   "attachments": {},',
    '    "cell_type": "markdown",',
    '    "metadata": {},',
    '    "source": [',
    '@@ -9,7 +8,6 @@',
    '    ]',
    '   },',
    'diff --git a/queries/repricing.sql b/queries/repricing.sql',
    'index 422e7884d..b82c9924a 100644',
    '--- a/queries/repricing.sql',
    '+++ b/queries/repricing.sql',
    '@@ -62,7 +62,7 @@ relevant_leads AS (',
    '         AND mod(cl.id, 7) = date_part(DAYOFWEEK, current_date)',
    ' ),',
]

filtered_diff = filter_diff_for_sql_and_py(diff_lines)
print("\n".join(filtered_diff))


diff --git a/queries/repricing.sql b/queries/repricing.sql
index 422e7884d..b82c9924a 100644
--- a/queries/repricing.sql
+++ b/queries/repricing.sql
@@ -62,7 +62,7 @@ relevant_leads AS (
         AND mod(cl.id, 7) = date_part(DAYOFWEEK, current_date)
 ),


In [27]:
import openai

def review_code_with_gpt4(code_diff, openai_api_key):
    """
    Sends the code diff to GPT-4 and receives feedback.
    
    :param code_diff: The code differences as a string.
    :param openai_api_key: Your OpenAI API key.
    :return: The feedback from GPT-4 as a string.
    """
    client = OpenAI(api_key = openai_api_key)
    try:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {
                    "role":"user", 
                    "content":f"Review the following github PR code changes:\n\n{code_diff}\n\nProvide detailed constructive feedback to improve the code given. When suggesting changes, please show the line before and after change."
                }
            ]
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"An error occurred: {e}")
        return "Failed to get feedback from GPT-4."

def split_diff_and_review(code_diff, openai_api_key, max_chars=10000):
    """
    Splits the diff into chunks based on a character count approximation to stay within token limits.
    """
    # Initialize variables
    feedbacks = []
    chunk = ""

    # Splitting the diff more accurately by character count
    for line in code_diff:
        # Check if adding the next line would exceed the max character count
        if len(chunk) + len(line) + 1 > max_chars:  # +1 for newline character
            # Review the current chunk
            feedback = review_code_with_gpt4(chunk, openai_api_key)
            feedbacks.append(feedback)
            # Start a new chunk
            chunk = line
        else:
            # Add the line to the current chunk
            chunk += f"\n{line}"

    # Don't forget to process the last chunk if it's not empty
    if chunk:
        feedback = review_code_with_gpt4(chunk, openai_api_key)
        feedbacks.append(feedback)

    return " ".join(feedbacks)

# Example usage
owner = "wkda"
repo = "ds-auto-pricing-service"
pr_number = 1231
github_token = os.getenv("GITHUB_API_KEY")  # Replace with your GitHub token if needed
openai_api_key = os.getenv("OPEN_AI_API_KEY")  # Replace with your OpenAI API key

print(f"Fetching PR #{pr_number} diff from {owner}/{repo}...")
pr_diff = fetch_pr_diff(owner, repo, pr_number, github_token)
pr_diff_filtered = filter_diff_for_sql_and_py(pr_diff.split("\n"))
print(pr_diff_filtered)
feedback = split_diff_and_review(pr_diff_filtered, openai_api_key)
print(feedback)


Fetching PR #1231 diff from wkda/ds-auto-pricing-service...
['diff --git a/queries/repricing.sql b/queries/repricing.sql', 'index 422e7884d..b82c9924a 100644', '--- a/queries/repricing.sql', '+++ b/queries/repricing.sql', '@@ -62,7 +62,7 @@ relevant_leads AS (', '         AND mod(cl.id, 7) = date_part(DAYOFWEEK, current_date)', ' ),', ' ', '-private_listing AS (', '+self_evaluations AS (', '     SELECT', '         cl.id,', '         pl.car_lead_hash,', '@@ -74,12 +74,12 @@ private_listing AS (', '         pl.make,', '         pl.model', '     FROM relevant_leads AS cl', '-    LEFT JOIN wkda_classifieds_private_listing.private_listing AS pl ON', '+    LEFT JOIN wkda_self_evaluation.self_evaluations AS pl ON', '         pl.car_lead_hash = cl.hash', '         AND pl.submitted_on IS NOT NULL', ' ),', ' ', '-private_listing_history AS (', '+self_evaluation_history AS (', '     SELECT', '         cl.id,', '         epl.car_lead_hash,', '@@ -96,14 +96,14 @@ private_listing_history AS (', '   

In [28]:
feedback

"This PR is about renaming variables and tables used in SQL queries. The refactoring appears clean and done in a consistent manner. Here are a few points that could be improved or are worth noting:\n\n1. Comment Specification: Add brief comments next to every change with a clear explanation of why that change is necessary, or what it accomplishes. This will help future readers. For instance:\n\n```\n-    JOIN wkda_classifieds_private_listing.private_listing AS pl ON pl.car_lead_hash = cl.hash\n+    JOIN wkda_self_evaluation.self_evaluations as self_evaluations ON cl.hash = self_evaluations.car_lead_hash // changed reference to new self evaluation table\n\n```\n2. Use of aliases: Use shorter aliases for tables. The current aliases are too long and hard to follow throughout the code. For example, change 'self_evaluations' to 'se' or 'eval'. \n\n```\n-    JOIN wkda_self_evaluation.self_evaluations as self_evaluations ON cl.hash = self_evaluations.car_lead_hash\n+    JOIN wkda_self_evaluat

In [ ]:
import openai

def review_code_with_gpt4(code_diff, openai_api_key):
    """
    Sends the code diff to GPT-4 and receives feedback.
    
    :param code_diff: The code differences as a string.
    :param openai_api_key: Your OpenAI API key.
    :return: The feedback from GPT-4 as a string.
    """
    client = OpenAI(api_key = openai_api_key)
    try:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {
                    "role":"user", 
                    "content":f"Review the following github PR code changes:\n\n{code_diff}\n\nProvide detailed constructive feedback to improve the code given. When suggesting changes, please show the line before and after change."
                }
            ]
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"An error occurred: {e}")
        return "Failed to get feedback from GPT-4."

def split_diff_and_review(code_diff, openai_api_key, max_chars=10000):
    """
    Splits the diff into chunks based on a character count approximation to stay within token limits.
    """
    # Initialize variables
    feedbacks = []
    chunk = ""

    # Splitting the diff more accurately by character count
    for line in code_diff:
        # Check if adding the next line would exceed the max character count
        if len(chunk) + len(line) + 1 > max_chars:  # +1 for newline character
            # Review the current chunk
            feedback = review_code_with_gpt4(chunk, openai_api_key)
            feedbacks.append(feedback)
            # Start a new chunk
            chunk = line
        else:
            # Add the line to the current chunk
            chunk += f"\n{line}"

    # Don't forget to process the last chunk if it's not empty
    if chunk:
        feedback = review_code_with_gpt4(chunk, openai_api_key)
        feedbacks.append(feedback)

    return " ".join(feedbacks)

# Example usage
owner = "wkda"
repo = "ds-retail-autopricer-service"
pr_number = 1231
github_token = os.getenv("GITHUB_API_KEY")  # Replace with your GitHub token if needed
openai_api_key = os.getenv("OPEN_AI_API_KEY")  # Replace with your OpenAI API key

print(f"Fetching PR #{pr_number} diff from {owner}/{repo}...")
pr_diff = fetch_pr_diff(owner, repo, pr_number, github_token)
pr_diff_filtered = filter_diff_for_sql_and_py(pr_diff.split("\n"))
print(pr_diff_filtered)
feedback = split_diff_and_review(pr_diff_filtered, openai_api_key)
print(feedback)
